<a href="https://colab.research.google.com/github/ndb796/Deep-Learning-Paper-Review-and-Practice/blob/master/code_practices/ResNet18_CIFAR10_Train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch

torch.cuda.is_available()

True

In [2]:
import torch
import random
import torch.backends.cudnn as cudnn

torch.manual_seed(0)
torch.cuda.manual_seed(0)
torch.cuda.manual_seed_all(0)
cudnn.benchmark = False
cudnn.deterministic = True
random.seed(0)

#### ResNet18 모델 정의 및 인스턴스 초기화

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
import torch.optim as optim
import os


# ResNet18을 위해 최대한 간단히 수정한 BasicBlock 클래스 정의
class BasicBlock(nn.Module):
    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()

        # 3x3 필터를 사용 (너비와 높이를 줄일 때는 stride 값 조절)
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes) # 배치 정규화(batch normalization)

        # 3x3 필터를 사용 (패딩을 1만큼 주기 때문에 너비와 높이가 동일)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes) # 배치 정규화(batch normalization)

        self.shortcut = nn.Sequential() # identity인 경우
        if stride != 1: # stride가 1이 아니라면, Identity mapping이 아닌 경우
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(planes)
            )

    def forward(self, x):
        out = F.leaky_relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x) # (핵심) skip connection
        out = F.leaky_relu(out)
        return out


# ResNet 클래스 정의
class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10):
        super(ResNet, self).__init__()
        self.in_planes = 64

        # 64개의 3x3 필터(filter)를 사용
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        self.linear = nn.Linear(512, num_classes)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks - 1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes # 다음 레이어를 위해 채널 수 변경
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.leaky_relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.avg_pool2d(out, 4)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out


# ResNet18 함수 정의
def ResNet18():
    return ResNet(BasicBlock, [2, 2, 2, 2])

#### 데이터셋(Dataset) 다운로드 및 불러오기

In [4]:
import torchvision
import torchvision.transforms as transforms

transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
])

train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=128, shuffle=True, num_workers=4)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=100, shuffle=False, num_workers=4)

Files already downloaded and verified
Files already downloaded and verified


#### 환경 설정 및 학습(Training) 함수 정의

In [5]:
device = 'cuda'

net = ResNet18()
net = net.to(device)
net = torch.nn.DataParallel(net)
cudnn.benchmark = True

learning_rate = 0.1
file_name = 'resnet18_cifar10.pt'

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=learning_rate, momentum=0.9, weight_decay=0.0002)


def train(epoch):
    print('\n[ Train epoch: %d ]' % epoch)
    net.train()
    train_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(train_loader):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()

        benign_outputs = net(inputs)
        loss = criterion(benign_outputs, targets)
        loss.backward()

        optimizer.step()
        train_loss += loss.item()
        _, predicted = benign_outputs.max(1)

        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()
        
        if batch_idx % 100 == 0:
            print('\nCurrent batch:', str(batch_idx))
            print('Current benign train accuracy:', str(predicted.eq(targets).sum().item() / targets.size(0)))
            print('Current benign train loss:', loss.item())

    print('\nTotal benign train accuarcy:', 100. * correct / total)
    print('Total benign train loss:', train_loss)


def test(epoch):
    print('\n[ Test epoch: %d ]' % epoch)
    net.eval()
    loss = 0
    correct = 0
    total = 0

    for batch_idx, (inputs, targets) in enumerate(test_loader):
        inputs, targets = inputs.to(device), targets.to(device)
        total += targets.size(0)

        outputs = net(inputs)
        loss += criterion(outputs, targets).item()

        _, predicted = outputs.max(1)
        correct += predicted.eq(targets).sum().item()

    print('\nTest accuarcy:', 100. * correct / total)
    print('Test average loss:', loss / total)

    state = {
        'net': net.state_dict()
    }
    if not os.path.isdir('checkpoint'):
        os.mkdir('checkpoint')
    torch.save(state, './checkpoint/' + file_name)
    print('Model Saved!')


def adjust_learning_rate(optimizer, epoch):
    lr = learning_rate
    if epoch >= 50:
        lr = 0.01
    if epoch >= 80:
        lr = 0.001
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

#### 학습(Training) 진행

* 대략 20번의 epoch 이후에도 85%가량의 test accuracy를 얻을 수 있습니다.

In [6]:
# for epoch in range(0, 200):
for epoch in range(0, 100):
    adjust_learning_rate(optimizer, epoch)
    train(epoch)
    test(epoch)


[ Train epoch: 0 ]

Current batch: 0
Current benign train accuracy: 0.1171875
Current benign train loss: 2.3607285022735596

Current batch: 100
Current benign train accuracy: 0.2578125
Current benign train loss: 1.808032512664795

Current batch: 200
Current benign train accuracy: 0.3515625
Current benign train loss: 1.7413809299468994

Current batch: 300
Current benign train accuracy: 0.3828125
Current benign train loss: 1.7162535190582275

Total benign train accuarcy: 32.712
Total benign train loss: 722.6255869865417

[ Test epoch: 0 ]

Test accuarcy: 42.26
Test average loss: 0.015493653297424316
Model Saved!

[ Train epoch: 1 ]

Current batch: 0
Current benign train accuracy: 0.421875
Current benign train loss: 1.6653252840042114

Current batch: 100
Current benign train accuracy: 0.40625
Current benign train loss: 1.4895802736282349

Current batch: 200
Current benign train accuracy: 0.46875
Current benign train loss: 1.4186780452728271

Current batch: 300
Current benign train accura


Current batch: 200
Current benign train accuracy: 0.90625
Current benign train loss: 0.3212822675704956

Current batch: 300
Current benign train accuracy: 0.9140625
Current benign train loss: 0.24651190638542175

Total benign train accuarcy: 87.316
Total benign train loss: 143.0538979023695

[ Test epoch: 13 ]

Test accuarcy: 85.61
Test average loss: 0.004288868509232998
Model Saved!

[ Train epoch: 14 ]

Current batch: 0
Current benign train accuracy: 0.875
Current benign train loss: 0.35085049271583557

Current batch: 100
Current benign train accuracy: 0.921875
Current benign train loss: 0.245141863822937

Current batch: 200
Current benign train accuracy: 0.8203125
Current benign train loss: 0.49161675572395325

Current batch: 300
Current benign train accuracy: 0.859375
Current benign train loss: 0.46590936183929443

Total benign train accuarcy: 88.096
Total benign train loss: 134.611326277256

[ Test epoch: 14 ]

Test accuarcy: 82.86
Test average loss: 0.005224328780174255
Model Sa


Total benign train accuarcy: 91.448
Total benign train loss: 97.2032902687788

[ Test epoch: 26 ]

Test accuarcy: 86.5
Test average loss: 0.004226913607120514
Model Saved!

[ Train epoch: 27 ]

Current batch: 0
Current benign train accuracy: 0.953125
Current benign train loss: 0.20508292317390442

Current batch: 100
Current benign train accuracy: 0.8984375
Current benign train loss: 0.3266514539718628

Current batch: 200
Current benign train accuracy: 0.8515625
Current benign train loss: 0.3513637185096741

Current batch: 300
Current benign train accuracy: 0.921875
Current benign train loss: 0.25584614276885986

Total benign train accuarcy: 91.722
Total benign train loss: 93.53790916502476

[ Test epoch: 27 ]

Test accuarcy: 86.16
Test average loss: 0.004372449205815792
Model Saved!

[ Train epoch: 28 ]

Current batch: 0
Current benign train accuracy: 0.953125
Current benign train loss: 0.17256397008895874

Current batch: 100
Current benign train accuracy: 0.9296875
Current benign tra


Current batch: 0
Current benign train accuracy: 0.9296875
Current benign train loss: 0.2162187546491623

Current batch: 100
Current benign train accuracy: 0.84375
Current benign train loss: 0.33655908703804016

Current batch: 200
Current benign train accuracy: 0.921875
Current benign train loss: 0.20337918400764465

Current batch: 300
Current benign train accuracy: 0.9453125
Current benign train loss: 0.1514262557029724

Total benign train accuarcy: 92.784
Total benign train loss: 81.06816720217466

[ Test epoch: 40 ]

Test accuarcy: 87.41
Test average loss: 0.004046460719406605
Model Saved!

[ Train epoch: 41 ]

Current batch: 0
Current benign train accuracy: 0.9375
Current benign train loss: 0.14971263706684113

Current batch: 100
Current benign train accuracy: 0.953125
Current benign train loss: 0.1324339210987091

Current batch: 200
Current benign train accuracy: 0.921875
Current benign train loss: 0.20378603041172028

Current batch: 300
Current benign train accuracy: 0.9296875
Cu


Current batch: 200
Current benign train accuracy: 0.9921875
Current benign train loss: 0.024426106363534927

Current batch: 300
Current benign train accuracy: 1.0
Current benign train loss: 0.0269133523106575

Total benign train accuarcy: 98.738
Total benign train loss: 15.41077784774825

[ Test epoch: 53 ]

Test accuarcy: 93.87
Test average loss: 0.002040966455079615
Model Saved!

[ Train epoch: 54 ]

Current batch: 0
Current benign train accuracy: 0.9921875
Current benign train loss: 0.017068086192011833

Current batch: 100
Current benign train accuracy: 0.9921875
Current benign train loss: 0.04874078184366226

Current batch: 200
Current benign train accuracy: 1.0
Current benign train loss: 0.005123523995280266

Current batch: 300
Current benign train accuracy: 0.96875
Current benign train loss: 0.10048957169055939

Total benign train accuarcy: 98.968
Total benign train loss: 13.324786766432226

[ Test epoch: 54 ]

Test accuarcy: 94.0
Test average loss: 0.002075999836996198
Model Sa


Total benign train accuarcy: 99.674
Total benign train loss: 4.532783621922135

[ Test epoch: 66 ]

Test accuarcy: 94.12
Test average loss: 0.0023109120693057777
Model Saved!

[ Train epoch: 67 ]

Current batch: 0
Current benign train accuracy: 1.0
Current benign train loss: 0.009072802029550076

Current batch: 100
Current benign train accuracy: 1.0
Current benign train loss: 0.008565113879740238

Current batch: 200
Current benign train accuracy: 1.0
Current benign train loss: 0.0034565720707178116

Current batch: 300
Current benign train accuracy: 0.984375
Current benign train loss: 0.02535809576511383

Total benign train accuarcy: 99.76
Total benign train loss: 3.8076833694940433

[ Test epoch: 67 ]

Test accuarcy: 94.02
Test average loss: 0.002320850771665573
Model Saved!

[ Train epoch: 68 ]

Current batch: 0
Current benign train accuracy: 1.0
Current benign train loss: 0.009167133830487728

Current batch: 100
Current benign train accuracy: 1.0
Current benign train loss: 0.0021654


Current batch: 100
Current benign train accuracy: 1.0
Current benign train loss: 0.001976940780878067

Current batch: 200
Current benign train accuracy: 1.0
Current benign train loss: 0.003992666956037283

Current batch: 300
Current benign train accuracy: 1.0
Current benign train loss: 0.0013467343524098396

Total benign train accuarcy: 99.878
Total benign train loss: 1.9182686845015269

[ Test epoch: 80 ]

Test accuarcy: 94.36
Test average loss: 0.0023069975052028894
Model Saved!

[ Train epoch: 81 ]

Current batch: 0
Current benign train accuracy: 1.0
Current benign train loss: 0.005372751969844103

Current batch: 100
Current benign train accuracy: 1.0
Current benign train loss: 0.005382718052715063

Current batch: 200
Current benign train accuracy: 1.0
Current benign train loss: 0.0032783967908471823

Current batch: 300
Current benign train accuracy: 1.0
Current benign train loss: 0.004379738122224808

Total benign train accuarcy: 99.936
Total benign train loss: 1.6014605103991926



Total benign train accuarcy: 99.958
Total benign train loss: 1.2180897002108395

[ Test epoch: 93 ]

Test accuarcy: 94.53
Test average loss: 0.0022803097198717294
Model Saved!

[ Train epoch: 94 ]

Current batch: 0
Current benign train accuracy: 1.0
Current benign train loss: 0.0018520503072068095

Current batch: 100
Current benign train accuracy: 1.0
Current benign train loss: 0.002982437377795577

Current batch: 200
Current benign train accuracy: 1.0
Current benign train loss: 0.001519093639217317

Current batch: 300
Current benign train accuracy: 0.9921875
Current benign train loss: 0.03016509860754013

Total benign train accuarcy: 99.954
Total benign train loss: 1.2053613415628206

[ Test epoch: 94 ]

Test accuarcy: 94.49
Test average loss: 0.0022907245201058686
Model Saved!

[ Train epoch: 95 ]

Current batch: 0
Current benign train accuracy: 1.0
Current benign train loss: 0.0021170713007450104

Current batch: 100
Current benign train accuracy: 1.0
Current benign train loss: 0.00